In [1]:
#Starter notebook taken from hiromu
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

In [16]:
tourney_result = pd.read_csv('WDataFiles_Stage1/WNCAATourneyCompactResults.csv')
tourney_seed = pd.read_csv('WDataFiles_Stage1/WNCAATourneySeeds.csv')
tourney_seed

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272
...,...,...,...
1403,2019,Z12,3416
1404,2019,Z13,3195
1405,2019,Z14,3200
1406,2019,Z15,3340


In [17]:
# deleting unnecessary columns
tourney_result = tourney_result.drop(['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], axis=1)
tourney_result

,Season,WTeamID,LTeamID
0,1998,3104,3422
1,1998,3112,3365
2,1998,3163,3193
3,1998,3198,3266
4,1998,3203,3208
...,...,...,...
1381,2019,3124,3234
1382,2019,3323,3390
1383,2019,3124,3332
1384,2019,3323,3163


In [18]:
tourney_result = pd.merge(tourney_result, tourney_seed, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Seed':'WSeed'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result = pd.merge(tourney_result, tourney_seed, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Seed':'LSeed'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result

,Season,WTeamID,LTeamID,WSeed,LSeed
0,1998,3104,3422,X02,X15
1,1998,3112,3365,W03,W14
2,1998,3163,3193,W02,W15
3,1998,3198,3266,Y07,Y10
4,1998,3203,3208,W10,W07
...,...,...,...,...,...
1381,2019,3124,3234,Y01,Y02
1382,2019,3323,3390,X01,X02
1383,2019,3124,3332,Y01,Z02
1384,2019,3323,3163,X01,W02


In [21]:
def get_seed(x):
    return int(x[1:3])

tourney_result['WSeed'] = tourney_result['WSeed'].map(lambda x: get_seed(x))
tourney_result['LSeed'] = tourney_result['LSeed'].map(lambda x: get_seed(x))
tourney_result

,Season,WTeamID,LTeamID,WSeed,LSeed
0,1998,3104,3422,2,15
1,1998,3112,3365,3,14
2,1998,3163,3193,2,15
3,1998,3198,3266,7,10
4,1998,3203,3208,10,7
...,...,...,...,...,...
1381,2019,3124,3234,1,2
1382,2019,3323,3390,1,2
1383,2019,3124,3332,1,2
1384,2019,3323,3163,1,2


In [22]:
season_result = pd.read_csv('WDataFiles_Stage1/WRegularSeasonCompactResults.csv')

In [23]:
season_win_result = season_result[['Season', 'WTeamID', 'WScore']]
season_lose_result = season_result[['Season', 'LTeamID', 'LScore']]
season_win_result.rename(columns={'WTeamID':'TeamID', 'WScore':'Score'}, inplace=True)
season_lose_result.rename(columns={'LTeamID':'TeamID', 'LScore':'Score'}, inplace=True)
season_result = pd.concat((season_win_result, season_lose_result)).reset_index(drop=True)
season_result

/Users/jonathanpiech/anaconda3/envs/ncaa/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,Season,TeamID,Score
0,1998,3104,91
1,1998,3163,87
2,1998,3222,66
3,1998,3307,69
4,1998,3349,115
...,...,...,...
214019,2019,3251,49
214020,2019,3179,79
214021,2019,3335,54
214022,2019,3144,45


In [24]:
season_score = season_result.groupby(['Season', 'TeamID'])['Score'].sum().reset_index()
season_score

,Season,TeamID,Score
0,1998,3102,1375
1,1998,3103,2008
2,1998,3104,2297
3,1998,3106,1286
4,1998,3108,1560
...,...,...,...
7350,2019,3462,1765
7351,2019,3463,1837
7352,2019,3464,1976
7353,2019,3465,1905


In [25]:
tourney_result = pd.merge(tourney_result, season_score, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Score':'WScoreT'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result = pd.merge(tourney_result, season_score, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
tourney_result.rename(columns={'Score':'LScoreT'}, inplace=True)
tourney_result = tourney_result.drop('TeamID', axis=1)
tourney_result

,Season,WTeamID,LTeamID,WSeed,LSeed,WScoreT,LScoreT
0,1998,3104,3422,2,15,2297,2010
1,1998,3112,3365,3,14,2116,2144
2,1998,3163,3193,2,15,2696,1950
3,1998,3198,3266,7,10,2337,1907
4,1998,3203,3208,10,7,1818,1996
...,...,...,...,...,...,...,...
1381,2019,3124,3234,1,2,2575,2555
1382,2019,3323,3390,1,2,2938,2393
1383,2019,3124,3332,1,2,2575,2725
1384,2019,3323,3163,1,2,2938,2726


In [26]:
tourney_win_result = tourney_result.drop(['Season', 'WTeamID', 'LTeamID'], axis=1)
tourney_win_result.rename(columns={'WSeed':'Seed1', 'LSeed':'Seed2', 'WScoreT':'ScoreT1', 'LScoreT':'ScoreT2'}, inplace=True)
tourney_win_result

,Seed1,Seed2,ScoreT1,ScoreT2
0,2,15,2297,2010
1,3,14,2116,2144
2,2,15,2696,1950
3,7,10,2337,1907
4,10,7,1818,1996
...,...,...,...,...
1381,1,2,2575,2555
1382,1,2,2938,2393
1383,1,2,2575,2725
1384,1,2,2938,2726


In [29]:
tourney_lose_result = tourney_win_result.copy()
tourney_lose_result['Seed1'] = tourney_win_result['Seed2']
tourney_lose_result['Seed2'] = tourney_win_result['Seed1']
tourney_lose_result['ScoreT1'] = tourney_win_result['ScoreT2']
tourney_lose_result['ScoreT2'] = tourney_win_result['ScoreT1']
tourney_lose_result

,Seed1,Seed2,ScoreT1,ScoreT2,Seed_diff,ScoreT_diff
0,15,2,2010,2297,-13,287
1,14,3,2144,2116,-11,-28
2,15,2,1950,2696,-13,746
3,10,7,1907,2337,-3,430
4,7,10,1996,1818,3,-178
...,...,...,...,...,...,...
1381,2,1,2555,2575,-1,20
1382,2,1,2393,2938,-1,545
1383,2,1,2725,2575,-1,-150
1384,2,1,2726,2938,-1,212


In [30]:
tourney_win_result['Seed_diff'] = tourney_win_result['Seed1'] - tourney_win_result['Seed2']
tourney_win_result['ScoreT_diff'] = tourney_win_result['ScoreT1'] - tourney_win_result['ScoreT2']
tourney_lose_result['Seed_diff'] = tourney_lose_result['Seed1'] - tourney_lose_result['Seed2']
tourney_lose_result['ScoreT_diff'] = tourney_lose_result['ScoreT1'] - tourney_lose_result['ScoreT2']

In [31]:
tourney_win_result['result'] = 1
tourney_lose_result['result'] = 0
tourney_result = pd.concat((tourney_win_result, tourney_lose_result)).reset_index(drop=True)
tourney_result

,Seed1,Seed2,ScoreT1,ScoreT2,Seed_diff,ScoreT_diff,result
0,2,15,2297,2010,-13,287,1
1,3,14,2116,2144,-11,-28,1
2,2,15,2696,1950,-13,746,1
3,7,10,2337,1907,-3,430,1
4,10,7,1818,1996,3,-178,1
...,...,...,...,...,...,...,...
2767,2,1,2555,2575,1,-20,0
2768,2,1,2393,2938,1,-545,0
2769,2,1,2725,2575,1,150,0
2770,2,1,2726,2938,1,-212,0


In [32]:
X_train = tourney_result.drop('result', axis=1)
y_train = tourney_result.result
X_train, y_train = shuffle(X_train, y_train)

In [33]:
logreg = LogisticRegression()
params = {'C': np.logspace(start=-5, stop=3, num=9)}
clf = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True)
clf.fit(X_train, y_train)
print('Best log_loss: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))

/Users/jonathanpiech/anaconda3/envs/ncaa/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/jonathanpiech/anaconda3/envs/ncaa/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jonathanpiech/anaconda3/envs/ncaa/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jonathanpiech/anaconda3/envs/ncaa/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jonathanpiech/anac

Best log_loss: -0.4481, with best C: 0.1


/Users/jonathanpiech/anaconda3/envs/ncaa/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jonathanpiech/anaconda3/envs/ncaa/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jonathanpiech/anaconda3/envs/ncaa/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jonathanpiech/anaconda3/envs/ncaa/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/jonathanpiech/anaconda3/envs/ncaa/lib/python3.7/site-packages/skl

In [34]:
test_df = pd.read_csv('WSampleSubmissionStage1_2020.csv')

In [35]:
test_df['Season'] = test_df['ID'].map(lambda x: int(x[:4]))
test_df['WTeamID'] = test_df['ID'].map(lambda x: int(x[5:9]))
test_df['LTeamID'] = test_df['ID'].map(lambda x: int(x[10:14]))
test_df

,ID,Pred,Season,WTeamID,LTeamID
0,2015_3106_3107,0.5,2015,3106,3107
1,2015_3106_3110,0.5,2015,3106,3110
2,2015_3106_3113,0.5,2015,3106,3113
3,2015_3106_3114,0.5,2015,3106,3114
4,2015_3106_3116,0.5,2015,3106,3116
...,...,...,...,...,...
10075,2019_3413_3417,0.5,2019,3413,3417
10076,2019_3413_3460,0.5,2019,3413,3460
10077,2019_3416_3417,0.5,2019,3416,3417
10078,2019_3416_3460,0.5,2019,3416,3460


In [36]:
test_df = pd.merge(test_df, tourney_seed, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
test_df.rename(columns={'Seed':'Seed1'}, inplace=True)
test_df = test_df.drop('TeamID', axis=1)
test_df = pd.merge(test_df, tourney_seed, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
test_df.rename(columns={'Seed':'Seed2'}, inplace=True)
test_df = test_df.drop('TeamID', axis=1)
test_df = pd.merge(test_df, season_score, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
test_df.rename(columns={'Score':'ScoreT1'}, inplace=True)
test_df = test_df.drop('TeamID', axis=1)
test_df = pd.merge(test_df, season_score, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
test_df.rename(columns={'Score':'ScoreT2'}, inplace=True)
test_df = test_df.drop('TeamID', axis=1)
test_df

,ID,Pred,Season,WTeamID,LTeamID,Seed1,Seed2,ScoreT1,ScoreT2
0,2015_3106_3107,0.5,2015,3106,3107,Y15,X13,1692,2221
1,2015_3106_3110,0.5,2015,3106,3110,Y15,Z14,1692,2027
2,2015_3106_3113,0.5,2015,3106,3113,Y15,Y03,1692,2174
3,2015_3106_3114,0.5,2015,3106,3114,Y15,Y11,1692,2089
4,2015_3106_3116,0.5,2015,3106,3116,Y15,Z10,1692,1809
...,...,...,...,...,...,...,...,...,...
10075,2019_3413_3417,0.5,2019,3413,3417,X15,W06,2132,2346
10076,2019_3413_3460,0.5,2019,3413,3460,X15,X13,2132,2246
10077,2019_3416_3417,0.5,2019,3416,3417,Z12,W06,1973,2346
10078,2019_3416_3460,0.5,2019,3416,3460,Z12,X13,1973,2246


In [37]:
test_df['Seed1'] = test_df['Seed1'].map(lambda x: get_seed(x))
test_df['Seed2'] = test_df['Seed2'].map(lambda x: get_seed(x))
test_df['Seed_diff'] = test_df['Seed1'] - test_df['Seed2']
test_df['ScoreT_diff'] = test_df['ScoreT1'] - test_df['ScoreT2']
test_df = test_df.drop(['ID', 'Pred', 'Season', 'WTeamID', 'LTeamID'], axis=1)
test_df

,Seed1,Seed2,ScoreT1,ScoreT2,Seed_diff,ScoreT_diff
0,15,13,1692,2221,2,-529
1,15,14,1692,2027,1,-335
2,15,3,1692,2174,12,-482
3,15,11,1692,2089,4,-397
4,15,10,1692,1809,5,-117
...,...,...,...,...,...,...
10075,15,6,2132,2346,9,-214
10076,15,13,2132,2246,2,-114
10077,12,6,1973,2346,6,-373
10078,12,13,1973,2246,-1,-273


In [38]:
test_preds = clf.predict_proba(test_df)[:, 1]

In [39]:
submission_df = pd.read_csv('WSampleSubmissionStage1_2020.csv')
submission_df['Pred'] = test_preds
submission_df

,ID,Pred
0,2015_3106_3107,0.245201
1,2015_3106_3110,0.343249
2,2015_3106_3113,0.031159
3,2015_3106_3114,0.192136
4,2015_3106_3116,0.209320
...,...,...
10075,2019_3413_3417,0.083445
10076,2019_3413_3460,0.351097
10077,2019_3416_3417,0.132301
10078,2019_3416_3460,0.475381
